In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 13.1 Interfacing Between pandas and Model Code

In [3]:
data = pd.DataFrame({'x0': [1,2,3,4,5], 'x1': [0.01, -0.01, 0.25, -4.1, 0], 
                    'x2': [-1.5, 0, 3.6, 1.3, -2]})

In [4]:
data

,x0,x1,x2
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [5]:
data.columns

Index(['x0', 'x1', 'x2'], dtype='object')

In [6]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [7]:
df2 = pd.DataFrame(data=data.values, columns=['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [9]:
df3 = data.copy()

In [10]:
df3

,x0,x1,x2
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
df3['string'] = list('abcde')

In [12]:
df3

,x0,x1,x2,string
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [13]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [14]:
model_cols = 'x0 x1'.split()

In [15]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [16]:
data['category'] = pd.Categorical(list('abaab'), categories=['a', 'b'])

In [17]:
data

,x0,x1,x2,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [18]:
dummies = pd.get_dummies(data['category'], prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,x2,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


There are some nuances to fitting certain statistical models with dummy variables. It
may be simpler and less error-prone to use Patsy (the subject of the next section)
when you have more than simple numeric columns.

# 13.2 Creating Model Descriptions with Patsy

Patsy is a Python library for describing statistical models (especially linear models)
with a small string-based “formula syntax,” which is inspired by (but not exactly the
same as) the formula syntax used by the R and S statistical programming languages.

Patsy is well supported for specifying linear models in statsmodels, so I will focus on
some of the main features to help you get up and running. Patsy’s formulas are a spe‐
cial string syntax that looks like:

y ~ x0 + x1


In [19]:
data.drop('category', axis=1, inplace=True)

In [20]:
data.columns = ['x0', 'x1', 'y']

In [21]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [22]:
import patsy

In [23]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [24]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [25]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [26]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [27]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

The Patsy objects can be passed directly into algorithms like **numpy.linalg.lstsq**,
which performs an ordinary least squares regression:

In [28]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

C:\Users\ABC\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [29]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [30]:
s = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [31]:
s

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## Data Transformations in Patsy Formulas

In [32]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1)+1)', data)

In [33]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [34]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [35]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [36]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [37]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

The patsy.build_design_matrices function can apply transformations to new outof-sample data using the saved information from the original in-sample dataset:

In [38]:
new_data = pd.DataFrame({'x0': [6, 7, 8, 9], 'x1': [3.1, -0.5, 0, 2.3], 'y': [1, 2, 3, 4]})

In [39]:
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [40]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [41]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

The plus symbol (+) in the context of Patsy formulas does not mean addition,
when you want to add columns from a dataset by name, you must wrap them in the
special I function:

In [42]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [43]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [44]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

## Categorical Data and Patsy

In [45]:
data = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'], 'key2': [0, 1, 0, 1, 0, 1, 0, 0],
                     'v1': [1, 2, 3, 4, 5, 6, 7, 8], 'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})

In [46]:
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [47]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [48]:
y

DesignMatrix with shape (8, 1)
    v2
  -1.0
   0.0
   2.5
  -0.5
   4.0
  -1.2
   0.2
  -1.7
  Terms:
    'v2' (column 0)

In [49]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [50]:
#If you omit the intercept from the model, then columns for each category value will be included in the model design matrix
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [51]:
y

DesignMatrix with shape (8, 1)
    v2
  -1.0
   0.0
   2.5
  -0.5
   4.0
  -1.2
   0.2
  -1.7
  Terms:
    'v2' (column 0)

In [52]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Numeric columns can be interpreted as categorical with the C function

In [53]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [54]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

When you’re using multiple categorical terms in a model, things can be more complicated, as you can include interaction terms of the form key1:key2, which can be
used, for example, in analysis of variance (ANOVA) models

In [55]:
data['key2'] = data['key2'].map({0:'zero', 1:'one'})

In [56]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [57]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [58]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [59]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [60]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.3 Introduction to statsmodels

**statsmodels** is a Python library for fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization. Statsmodels contains more “classical” frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.

Some kinds of models found in statsmodels include:
* Linear models, generalized linear models, and robust linear models
* Linear mixed effects models
* Analysis of variance (ANOVA) methods
* Time series processes and state space models
* Generalized method of moments


## Estimating Linear Models

In [68]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [69]:
np.random.seed(12345)

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * np.random.randn(size)

N = 100

X = np.c_[dnorm(0,0.2,N), dnorm(0,0.4,N), dnorm(0,0.6,N)]
eps = dnorm(0,0.1,N)
beta = [0.1,0.3,0.5]

y = np.dot(X,beta) + eps

In [70]:
X[:5]

array([[-0.09154805, -0.99020862,  0.87334311],
       [ 0.21418997, -0.35578165, -0.44025243],
       [-0.23230006, -0.02065862,  0.23963091],
       [-0.24853015, -0.58755511, -0.44724086],
       [ 0.8791238 , -0.30520574, -0.90522006]])

In [71]:
y[:5]

array([ 0.68297794, -0.7514007 , -0.02922261, -0.53409808, -0.33607307])

In [72]:
x_model = sm.add_constant(X)

In [74]:
x_model[:5]

array([[ 1.        , -0.09154805, -0.99020862,  0.87334311],
       [ 1.        ,  0.21418997, -0.35578165, -0.44025243],
       [ 1.        , -0.23230006, -0.02065862,  0.23963091],
       [ 1.        , -0.24853015, -0.58755511, -0.44724086],
       [ 1.        ,  0.8791238 , -0.30520574, -0.90522006]])

In [75]:
model = sm.OLS(y, X)

In [76]:
results = model.fit()

In [77]:
results.params

array([0.21067788, 0.20574317, 0.50054902])

In [80]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.589
Model:                            OLS   Adj. R-squared (uncentered):              0.576
Method:                 Least Squares   F-statistic:                              46.26
Date:                Sat, 01 Aug 2020   Prob (F-statistic):                    1.20e-18
Time:                        21:19:51   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [92]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [93]:
data['y'] = y

In [95]:
data[:5]

,col0,col1,col2,y
0,-0.091548,-0.990209,0.873343,0.682978
1,0.214190,-0.355782,-0.440252,-0.751401
2,-0.232300,-0.020659,0.239631,-0.029223
3,-0.248530,-0.587555,-0.447241,-0.534098
4,0.879124,-0.305206,-0.905220,-0.336073


In [96]:
results = smf.ols('y ~ col0 + col1 + col2', data).fit()

In [97]:
results.params

Intercept    0.033559
col0         0.207691
col1         0.207931
col2         0.508549
dtype: float64

In [98]:
results.tvalues

Intercept     0.952188
col0          2.767758
col1          3.662977
col2         10.786058
dtype: float64

In [99]:
results.predict(data[:5])

0    0.252788
1   -0.219824
2    0.102881
3   -0.367674
4   -0.307667
dtype: float64

## Estimating Time Series Processes

In [100]:
init_x = 4

In [101]:
import random

In [102]:
values = [init_x] * 2

In [104]:
N = 1000

In [105]:
b0 = 0.8

In [106]:
b1 = -0.4

In [107]:
noise = dnorm(0,0.1,N)

In [108]:
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [109]:
MAX_LAGS = 5

In [110]:
model = sm.tsa.AR(values)

C:\Users\ABC\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


In [111]:
results = model.fit(MAX_LAGS)

In [112]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

# 13.4 Introduction to scikit-learn

In [113]:
train = pd.read_csv('titanic/train.csv')

In [114]:
test = pd.read_csv('titanic/test.csv')

In [116]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [123]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [125]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [126]:
impute_value = train['Age'].median()

In [127]:
train['Age'] = train['Age'].fillna(impute_value)

In [128]:
test['Age'] = test['Age'].fillna(impute_value)

In [135]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [136]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [137]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [138]:
X_train = train[predictors].values
y_train = train['Survived'].values
X_test = test[predictors].values

In [142]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [143]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [144]:
from sklearn.linear_model import LogisticRegression

In [145]:
model = LogisticRegression()

In [146]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [147]:
y_predict = model.predict(X_test)

In [149]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [151]:
from sklearn.linear_model import LogisticRegressionCV

In [153]:
model_cv = LogisticRegressionCV(10)

In [154]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [155]:
from sklearn.model_selection import cross_val_score

In [156]:
model_cv = LogisticRegressionCV(10)

In [157]:
scores = cross_val_score(model, X_train, y_train, cv=4)

In [158]:
scores

array([0.71748879, 0.79820628, 0.77130045, 0.78828829])